##  klue-roberta-model
- KLUE는 최초의 Korean Bench Mark Dataset으로 공식 github와 그에 대한 논문이 존재한다. 확인해본 결과, klue-roberta-model은 더 많은 데이터를 기반으로 학습하였으며 vocab size도 32,000여개로 충분하다.

## Data Augmentation
- Papago api와 연동해 크롤링 진행한 `premise`와 `hypothesis`의 한글재번역버전을 추가데이터로 사용
- KOR- > EN -> KOR 과정을 거침

In [ ]:
import pandas as pd
from glob import glob
import os
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from google import colab
colab.drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# go to file
os.chdir('/content/drive/MyDrive/dacon')

In [ ]:
train = pd.read_csv("train_data.csv")
test= pd.read_csv("test_data.csv")
submission = pd.read_csv("sample_submission.csv")

In [ ]:
train.drop('index', axis = 1,inplace = True)

In [ ]:
train.shape

(24998, 3)

In [ ]:
train[:10]

,premise,hypothesis,label
0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral
5,이번 증설로 코오롱인더스트리는 기존 생산량 7만7000톤에서 1만6800톤이 늘어나...,코오롱 인더스트리는 총 9만 3800톤의 생산 능력을 확보했다.,entailment
6,자신뿐만 아니라 남을 돕고자 하는 청년의 꿈과 열정에 모두가 주목하고 있다.,모든 청년은 꿈과 열정을 가지고 있다.,neutral
7,시대상황을 고려하는 현명한 시청태도가 요구되다.,시청태도에 특별한 주의점은 없다.,contradiction
8,사진과 차이없는 아기자기한 실내소품들과 분위기가 멋졌습니다.,아기자기한 실내소품들은 사진에서 본 것과 차이가 있었습니다.,contradiction
9,"빠른 답장과 간편한 체크인, 깨끗한 집 좋았어요",체크인이 복잡했어요.,contradiction


In [ ]:
premise_trans= pd.read_csv("premise_translate_final.csv")
hypo_trans= pd.read_csv("hypothesis_translate_final.csv")

In [ ]:
only = hypo_trans[['kor_hypothesis']]
merge = pd.concat([premise_trans,only],axis=1) #열 추가

In [ ]:
kor_merge=merge[['index','kor_premise','kor_hypothesis','label']] #열 지정

In [ ]:
kor_merge.drop('index', axis = 1,inplace = True) #인덱스 중복 방지

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
kor_merge.columns = ['premise','hypothesis','label']

In [ ]:
train_final = pd.concat([train,kor_merge])
train_final=train_final.reset_index(drop=True) #인덱스 초기화

In [ ]:
train = train_final[24998:]  #번역된 데이터(premise+hypo)만 사용

In [ ]:
#dev 추가
dev_data = pd.read_json("klue-nli-v1.1_dev.json")

In [ ]:
dev_data = dev_data[['premise','hypothesis','gold_label']]
dev_data .columns = ['premise','hypothesis','label']
train = pd.concat([train,dev_data])

In [ ]:
train.reset_index(drop=True)  #추가해서 약 2만8천개 (3천개 추가)

,premise,hypothesis,label
0,씨름은 예로부터 전해 내려오는 남자들의 대표적인 놀이로 넓고 평평한 백사장이나 마당...,씨름에 나오는 여자들의 놀이입니다.,contradiction
1,삼성은 자작극을 벌인 2명에 대해 형사고발 등 법적 대응을 검토하고 있다고 밝혔지만...,그들 자신의 연극을 만든 세 사람이 있었다.,contradiction
2,이를 위해 예측형 범죄예방시스템이 구축되고 고도화된다.,예측형 범죄예방 시스템 구축과 고도화가 목적이기 때문이다.,entailment
3,광주광역시가 재개발 정비사업 원주민 종합대책을 마련하는 등 원주민 보호에 적극 나섰다.,그 원주민들은 포괄적인 조치에 만족했습니다.,neutral
4,소비자와 직원들의 사랑을 받는 기업으로 정말 오래 지속하고 싶다면 이런 상황에서 책...,이런 상황에서 책임감을 드러내는 기업은 극소수다.,neutral
...,...,...,...
27993,흔히 비자림로라고 불리는 지방도 제1112호선을 넓히는 공사가 1년만에 재개되었다가...,지방도 제1112호선을 넓히는 공사는 중단없이 마무리 되었다.,contradiction
27994,흔히 비자림로라고 불리는 지방도 제1112호선을 넓히는 공사가 1년만에 재개되었다가...,지방도 제1112호선을 넓히는 공사가 중단된 건 세 번째이다.,neutral
27995,흔히 비자림로라고 불리는 지방도 제1112호선을 넓히는 공사가 1년만에 재개되었다가...,지방도 제1112호선은 흔히 비자림로라고 불린다.,entailment
27996,흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.,비흡연자는 발코니 있는 방이 필요없습니다.,neutral


## 전처리 - 정규식으로 특수문자 제거

In [ ]:
train['premise'] = train['premise'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]', '')
test['premise'] = test['premise'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]", "")
train['hypothesis'] = train['hypothesis'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]', '')
test['hypothesis'] = test['hypothesis'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [ ]:
train

## Tokenizing

In [ ]:
!pip install transformers

In [ ]:
import os
import random
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer

In [ ]:
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

- SEED 고정, GPU설정

In [ ]:
def seed_everything(seed:int = 1004):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(42)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
from sklearn.model_selection import train_test_split
train, eval = train_test_split(train, test_size=0.2, shuffle=True, stratify=train['label'])

In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')
max_seq_len = 128  # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids = [], [], []

    for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
        encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len, pad_to_max_length=True)

        input_ids.append(encoding_result['input_ids'])
        attention_masks.append(encoding_result['attention_mask'])
        token_type_ids.append(encoding_result['token_type_ids'])

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    return (input_ids, attention_masks, token_type_ids)

In [ ]:
X_train = convert_examples_to_features(train['premise'], train['hypothesis'],  max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/22398 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 22398/22398 [00:03<00:00, 6994.63it/s]


In [ ]:
length = []
for attention in X_train[1]:
  length.append(np.count_nonzero(attention))
print("학습 데이터 평균 단어 길이 : {}".format(np.mean(length)))
print("학습 데이터 최대 단어 길이 : {}".format(np.max(length)))

학습 데이터 평균 단어 길이 : 36.53540494687026
학습 데이터 최대 단어 길이 : 102


In [ ]:
X_valid = convert_examples_to_features(eval['premise'], eval['hypothesis'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/5600 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 5600/5600 [00:00<00:00, 7172.38it/s]


In [ ]:
val_length = []
for attention in X_valid[1]:
  val_length.append(np.count_nonzero(attention))
print("검증 데이터 평균 단어 길이 : {}".format(np.mean(val_length)))
print("검증 데이터 최대 단어 길이 : {}".format(np.max(val_length)))

검증 데이터 평균 단어 길이 : 36.78821428571428
검증 데이터 최대 단어 길이 : 91


In [ ]:
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    0 10559  2116  1235  2069  4758  2259   575  2178  2062   831  8198
   575  2073  1504  3771  2116  4041 14223  3771 23548   575 28674     2
  3982  2073  3619 14223  2079  3771 12190     2     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
train_label = train['label'].tolist()
val_label = eval['label'].tolist()

In [ ]:
from sklearn import preprocessing
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

y_train = idx_encode.transform(train_label) # 주어진 고유한 정수로 변환
y_val = idx_encode.transform(val_label) # 고유한 정수로 변환

In [ ]:
label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print(label_idx)
print(idx_label)

{'contradiction': 0, 'entailment': 1, 'neutral': 2}
{0: 'contradiction', 1: 'entailment', 2: 'neutral'}


## Modeling

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import TruncatedNormal
from transformers import TFAutoModel

In [ ]:
class Klue_RobertaClassifier(Model):
    def __init__(self, model_name):
        super(Klue_RobertaClassifier, self).__init__()
        self.bert = TFAutoModel.from_pretrained("klue/roberta-large", num_labels=3, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(rate=0.5)
        self.classifier = Dense(3, kernel_initializer=TruncatedNormal(0.02),activation='softmax') #가중치 초기화, 활성화함수

    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        input_ids, attention_mask, token_type_ids=inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output, training=training)
        prediction = self.classifier(pooled_output)

        return prediction

In [ ]:
model = Klue_RobertaClassifier("klue/roberta-large")

In [ ]:
from transformers import AdamWeightDecay
optimizer = AdamWeightDecay(1e-5, weight_decay_rate=1e-4)  #과적합 방지
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001,
    mode = 'max',
    patience=2)

model.fit(
    X_train, y_train, epochs=3, batch_size = 8, validation_data=(X_valid, y_val),
    callbacks = [early_stopping])

Epoch 1/3
2800/2800 [==============================] - 1500s 518ms/step - loss: 0.6460 - accuracy: 0.7335 - val_loss: 0.4970 - val_accuracy: 0.8120
Epoch 2/3
2800/2800 [==============================] - 1446s 517ms/step - loss: 0.3587 - accuracy: 0.8744 - val_loss: 0.5200 - val_accuracy: 0.8298
Epoch 3/3
2800/2800 [==============================] - 1445s 516ms/step - loss: 0.2129 - accuracy: 0.9314 - val_loss: 0.5374 - val_accuracy: 0.8259


## x_test 최종 예측

In [ ]:
X_test = convert_examples_to_features(test['premise'], test['hypothesis'],max_seq_len=128, tokenizer=tokenizer)

  0%|          | 0/1666 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1666/1666 [00:00<00:00, 4926.24it/s]


In [ ]:
test_length = []
for attention in X_test[1]:
  test_length.append(np.count_nonzero(attention))
print("평가 데이터 평균 단어 길이 : {}".format(np.mean(test_length)))
print("평가 데이터 최대 단어 길이 : {}".format(np.max(test_length)))

평가 데이터 평균 단어 길이 : 39.33613445378151
평가 데이터 최대 단어 길이 : 80


In [ ]:
test_label = test['label'].tolist()

In [ ]:
pred = model.predict(X_test)
result = [np.argmax(val) for val in pred]
out = [list(label_idx.keys())[_] for _ in result]

In [ ]:
out[:3]

['contradiction', 'neutral', 'entailment']

In [ ]:
test['label'] = out
test[:20]

,index,premise,hypothesis,label
0,0,다만 조금 좁아서 케리어를 펼치기 불편합니다,케리어를 펼치기에 공간이 충분했습니다,contradiction
1,1,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요,시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요,neutral
2,2,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다,무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다,entailment
3,3,몇 번을 다시봐도 볼 때마다 가슴이 저민다,다시 봤을때는 무덤덤했다,contradiction
4,4,8월 중에 입주신청을 하면 청년은 9월 신혼부부는 10월부터 입주가 가능하다,8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다,contradiction
5,5,마지막에 존트라볼타 지하철에서 혼자 담배 무는 장면 압권이네요,존트라볼타 지하철에서는 원칙상 담배를 피면 안된다,neutral
6,6,18일 귀국이라 발인도 지켜드리지 못해 더욱 죄송할 따름입니다,18일 배를 타고 여행을 떠났습니다,neutral
7,7,2003년 변정수 씨가 교통사고로 죽었다는 이야기가 돌았지만 사실무근이었다,2003년 변정수 씨는 교통사고를 당했다,contradiction
8,8,성룡의 초기 전성기를 만들어준 또 하나의 역작,성룡은 초기에 전성기를 맞았다,entailment
9,9,2015년 10월 임관해 사건 당시 대위였던 손씨는 수개월 뒤인 2018년 7월에 ...,손씨가 전역한 이후에 사건이 일어났다,contradiction


In [ ]:
test['label'].value_counts() 

neutral          602
entailment       547
contradiction    517
Name: label, dtype: int64

In [ ]:
submission['label']=out
submission.to_csv("dev추가_ONLYTranslate.csv", index=False)

In [ ]:
test

In [ ]:
pred_df = pd.DataFrame(pred)
pred_df

In [ ]:
pred_df.to_csv("dev추가_ONLYTranslate-pred값.csv",index = False)